In [1]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib notebook
import os
import sys
import pickle
from struct import *
import pandas as pd
from sklearn.model_selection import train_test_split
import data_utils
from data_utils import *
from Constants import *
from model import Net
from IPython.display import clear_output
import seaborn as sns
sns.set_style("white")
import warnings
warnings.filterwarnings("ignore")
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.nn import Linear, LSTM, GRU, Conv1d, Conv2d, Dropout, MaxPool2d, BatchNorm1d, BatchNorm2d, CrossEntropyLoss, MSELoss, BCELoss
from torch.nn.functional import relu, elu, relu6, sigmoid, tanh, softmax
from torch.nn.utils.weight_norm import weight_norm
from sklearn import preprocessing
import shap
import joblib

In [2]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
use_cuda = torch.cuda.is_available()
print("Running GPU.") if use_cuda else print("No GPU available.")
print(device)

No GPU available.
cpu


## Define functions

In [3]:
def randnorm(n):
    return np.random.normal(loc = 0, scale = 1, size = n).astype('float32')

# Function to get label
def get_labels(batch):
    #print("batch:", Variable(torch.from_numpy(batch['ts']).long()))
    return Variable(torch.from_numpy(batch['ts']).long())

# Function to get input
def get_input(batch):
    return {'x_img': get_variable(Variable(torch.from_numpy(batch['img'])))}
    #return {feat: Variable(torch.from_numpy(batch[feat])) for feat in FEATS}

def get_variable(x):
    """ Converts tensors to cuda, if available. """
    if use_cuda:
        return x.cuda()
    return x

def get_numpy(x):
    if use_cuda:
        return x.cpu().data.numpy()
    return x.data.numpy()

def accuracy(ys, ts):
    predictions = torch.max(ys, 1)[1]
    correct_prediction = torch.eq(predictions, ts)
    return torch.mean(correct_prediction.float())

def get_targets(batch):
    #print("target:", Variable(torch.from_numpy(batch['ts']).long()))
    return Variable(torch.FloatTensor(batch['ts']).long())

## Load the data
When predicting user AMPS error, each model should be specific to the individual.
The device and participant are set in Constants.py.

In [4]:
import pandas as pd
x = pd.read_csv('/data/Isabella/thesis_spring2022/NN/models/errors/error_totals.csv')
y = pd.read_csv('/data/AMPs/tagging_data_second-round.csv')
print(sum(x.num_errors),len(y))

1260 1427


In [5]:
# Upload the error data
errors_ = pd.read_csv('/data/Isabella/thesis_spring2022/NN/data_prep/error-tags_full.csv', index_col=[0])

errors_.loc[:, 'subID':'Track'] = errors_.loc[:, 'subID':'Track'].astype(str)
errors_.loc[:, 'HMD'] = errors_.loc[:, 'HMD'].astype(str)
errors_.loc[:, 'offset'] = errors_.loc[:, 'offset'].astype(bool)

# Determine whether the type of error is a motor error (and not a process error)
errors_['motor_error'] = False
for idx, row in errors_.iterrows():
    if row.error_type_value1=='1' or row.error_type_value2=='1' or row.error_type_value3=='1' or row.error_type_value4=='1' or row.error_type_value5=='1':
        errors_.loc[idx, 'motor_error'] = True
    
errors_ = errors_[errors_.HMD==DEVICE]
errors_ = errors_[errors_.motor_error==True]

errors_.reset_index(drop=True, inplace=True)

assert all(errors_['offset'] == True), "The timestamps of tagged errors have not been adjusted."

In [6]:
errors_.head()

,subID,task,Video,Track,Timestamp,errors_in_frame,time_to_prev_error,time_to_next_error,value1,error_type_value1,...,error_subtype_value4,value4b,value5,error_type_value5,error_subtype_value5,value5b,HMD,offset,Timestamp_adjusted,motor_error
0,P5,cereal,P5_cereal_2_merged_varjo.mp4,error frame,7.84,2,,-4,31,2,...,,,,,,,varjo,True,8.403196,True
1,P5,cereal,P5_cereal_2_merged_varjo.mp4,error frame,11.84,1,4,-7.52,13,1,...,,,,,,,varjo,True,12.403196,True
2,P5,cereal,P5_cereal_2_merged_varjo.mp4,error frame,19.36,2,7.52,-5.64,13,1,...,,,,,,,varjo,True,19.923196,True
3,P5,cereal,P5_cereal_2_merged_varjo.mp4,error frame,25.00,2,5.64,-7.36,6,1,...,,,,,,,varjo,True,25.563196,True
4,P5,cereal,P5_cereal_2_merged_varjo.mp4,error frame,32.36,1,7.36,-1.12,1,1,...,,,,,,,varjo,True,32.923196,True


In [7]:
# Read in the output data from event detection
sequences = pd.read_csv('/data/Isabella/thesis_spring2022/event_detect_out_final/all_sequences_varjo.csv')

#sequences.loc[(sequences.event=='sac')&(sequences.has_blink==1)] = 'blink'

# Filter data for error prediction
events_df = sequences[sequences.HMD==DEVICE]

In [8]:
# remove all loss, noise, and other events
events_df.drop(events_df.loc[(events_df.event=='noise')&(events_df.event=='loss')&(events_df.event=='other')].index.tolist(),inplace=True)

# Reset indices
events_df.reset_index(drop=True, inplace=True)

In [9]:
# Encode event as index using EVENT_DICT as defined in Constants.py
events_df['event'] = np.where(events_df.event=='fix', EVENT_DICT['fix'], np.where(events_df.event=='sac',EVENT_DICT['sac'],np.where(events_df.event=='smp',EVENT_DICT['smp'],np.where(events_df.event=='blink',EVENT_DICT['blink'],EVENT_DICT['other']))))

In [10]:
events_df = events_df.loc[:,'HMD':]
events_df.head()

,HMD,rate,eye,task,subID,VL,event,start_i,end_i,start_s,...,calculus_error,carpenter_error,P_nonfix,P_fix,P_ff,P_smp,P_sac,P_blink,has_blink,UID
0,varjo,200,right,cereal,P19,CVL,2,0,25,1.081565,...,0.366132,1.527210,0.534494,0.465506,5.470863e-11,0.465506,0.384207,0.150287,0,varjoP19cerealright
1,varjo,200,right,cereal,P19,CVL,0,26,49,1.211610,...,0.230767,2.684010,0.059622,0.940378,8.531830e-01,0.087195,0.057516,0.002106,0,varjoP19cerealright
2,varjo,200,right,cereal,P19,CVL,1,50,56,1.326648,...,0.732365,0.474248,0.728794,0.271206,2.022088e-01,0.068997,0.000396,0.728398,0,varjoP19cerealright
3,varjo,200,right,cereal,P19,CVL,4,57,59,1.361658,...,NaN,NaN,0.667140,0.332860,3.218246e-01,0.011035,0.000000,0.000000,0,varjoP19cerealright
4,varjo,200,right,cereal,P19,CVL,0,60,205,1.381670,...,0.913183,24.923594,NaN,NaN,NaN,NaN,NaN,NaN,0,varjoP19cerealright


In [12]:
# upload the experiment start and end times
offsets = pd.read_csv('/data/Isabella/thesis_spring2022/NN/data_prep/offsets.csv', index_col=[0])

In [13]:
offsets.head()

,HMD,subID,task,exper_start(s),exper_end(s)
0,varjo,P14,cereal,2.262060,175.435060
1,varjo,P14,sandwich,-0.044110,502.057489
2,varjo,P23,cereal,0.794094,189.215660
3,varjo,P23,sandwich,1.659792,948.839357
4,varjo,P19,cereal,2.963828,137.798528


In [14]:
# upload the differences between gaze data and videos
gaze_dur_diff = pd.read_csv('/data/Isabella/thesis_spring2022/NN/data_prep/gaze_dur_diff.csv', index_col=[0])
gaze_dur_diff.head()

,HMD,task,subID,type,lum_path,dur(s),min_lum_roc,min_lum_frame,min_lum_time(s),data_dur(s),diff(s)
0,varjo,cereal,P14,full,/data/AMPs/second-round/avg-lum-per-frame/P14_...,172.733166,-0.621721,206.0,6.866660,172.757628,-0.024462
2,varjo,cereal,P23,full,/data/AMPs/second-round/avg-lum-per-frame/P23_...,188.033152,-0.459796,201.0,6.699994,188.056043,-0.022891
4,varjo,cereal,P19,full,/data/AMPs/second-round/avg-lum-per-frame/P19_...,137.433200,-0.376091,174.0,5.799994,137.429682,0.003519
6,varjo,cereal,P2,full,/data/AMPs/second-round/avg-lum-per-frame/P2_c...,121.699882,-0.389706,193.0,6.433327,121.715036,-0.015154
8,varjo,cereal,P9,full,/data/AMPs/second-round/avg-lum-per-frame/P9_c...,123.899880,-0.535024,210.0,6.999993,123.907213,-0.007332


## Run the model

In [20]:
results = pd.DataFrame({})
    
for subID in events_df.subID.unique().tolist():
    
    labels = []
    features = []
    
    temp_ = events_df[events_df.subID == subID]
    temp_.reset_index(drop=True, inplace=True)

    errors = errors_[errors_.subID==subID]
    errors.reset_index(drop=True, inplace=True)

    for eye in temp_.eye.unique():

        success = []

        for task in temp_.task.unique().tolist():

            # Only include data for which there are labelled errors
            try:
                exper_start = offsets.loc[(offsets.subID==subID) & (offsets.task==task), 'exper_start(s)'].values[0]
                exper_end = offsets.loc[(offsets.subID==subID) & (offsets.task==task), 'exper_end(s)'].values[0]
                dur_diff = np.abs(gaze_dur_diff.loc[(gaze_dur_diff.subID==subID) & (gaze_dur_diff.task==task), 'diff(s)']).values[0]
                success.append(task)
            except:
                continue

            temp = temp_.loc[(temp_.task == task) & (temp_.eye == eye)]
            temp.reset_index(drop=True, inplace=True)

            error_t = errors[errors.task==task].Timestamp_adjusted.to_numpy()

            if len(error_t) == 0:
                continue



            for i in range(0, len(temp)-WINDOW, STEP):
                # Only add the sequence if it falls within the experiment time
                if temp.loc[i, 'start_s'] < exper_start or temp.loc[i+WINDOW-1, 'end_s'] > exper_end:
                    continue

                subID = temp.loc[i,'subID']
                VL = temp.loc[i,'VL']

                # Only label as error if all events fall within 2 seconds prior of error timestamp 
                #  and 5 seconds after errors timestamp -/+ the absolute value of duration difference between
                #  gaze video and gaze data abs(gaze_dur_diff).
                start_seq = temp.loc[i,'start_s']-3-dur_diff
                end_seq = temp.loc[i+WINDOW-1,'end_s']+2+dur_diff
                has_error = [(e >= start_seq and e <= end_seq) for e in error_t]
                error = np.any(has_error)

                if len(error_t) == 0 and CLASSIFIER == 'error':
                    continue

                # Event features to include
                event = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'event'].to_numpy())
                duration = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'duration'].to_numpy())
                amplitude = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'amplitude'].to_numpy())
                dispersion = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'duration'].to_numpy())
                avg_iss = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'avg_iss'].to_numpy())
                max_iss = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'max_iss'].to_numpy())
                carpenter_error = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'carpenter_error'].to_numpy())
                calculus_error = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'calculus_error'].to_numpy())
                has_blink = np.nan_to_num(temp.loc[i:i+WINDOW-1, 'has_blink'].to_numpy())
                random = np.random.normal(size=(1,WINDOW)).reshape((WINDOW,))

                # Sequence features to include (same across each event)

                # number of blinks that occur during the sequence
                num_blinks = [event.tolist().count(EVENT_DICT['blink'] + sum(has_blink))/WINDOW]*WINDOW # blink ratio

                # number of errors that happen within time range
                #num_errors = [sum(has_error)]*WINDOW

                # time from previous error to beginning of sequence
                prev_ = np.where(error_t < start_seq)[0]
                if len(prev_) > 0:
                    prev_error = [start_seq-error_t[max(prev_)]]*WINDOW
                else:
                    prev_error = [0.0]*WINDOW

                # time to next error from end of sequence
                next_ = np.where(error_t > end_seq)[0]
                if len(next_) > 0:
                    next_error = [error_t[min(next_)]-end_seq]*WINDOW
                else:
                    next_error = [0.0]*WINDOW

                # total duration of sequence
                total_dur = [temp.loc[i+WINDOW-1,'end_s']-temp.loc[i,'start_s']]*WINDOW

                # zero the non-relevant features depending on event (all blink feats)
                fixi = np.where(event==EVENT_DICT['fix'])[0]
                saci = np.where(event==EVENT_DICT['sac'])[0]
                smpi = np.where(event==EVENT_DICT['smp'])[0]
                blinki = np.where(event==EVENT_DICT['blink'])[0]
                duration[blinki], amplitude[blinki], dispersion[blinki], avg_iss[blinki], max_iss[blinki], carpenter_error[blinki], calculus_error[blinki] = 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0

                
                # labels of shape 1 x 4
                labels.append([subID, VL, task, error])
                # features of shape num_feats x window_size
                features.append(np.array([event,
                                          duration,
                                          amplitude,
                                          dispersion,
                                          avg_iss,
                                          max_iss,
                                          carpenter_error,
                                          calculus_error,
                                          #num_blinks,
                                          prev_error,
                                          next_error,
                                          total_dur
                                         ]).flatten())

    avg_tot_dur = sum([features[i][-1] for i in range(len(features))])/len(features)


    ## CREATE DATA
    feature_headers = []
    for feat in FEATS:
        feature_headers += [str(feat+str(i+1)) for i in range(WINDOW)]
    label_headers = LABELS

    # Combine into one dataframe
    Y = pd.DataFrame(data=np.array(labels), columns=label_headers)
    X = pd.DataFrame(data=np.array(features), columns=feature_headers)

    df = pd.concat([Y,X],axis=1)


    ## SPLIT INTO TRAIN AND TEST
    # Convert error column to boolean (1=True, 0=False)
    df['error'] = np.where(df.error=='True',1,0)

    # ensure that there are same proportion of errors in train and test
    positive = df[df[CLASSIFIER]==True]
    negative = df[df[CLASSIFIER]==False]

    X_positive = positive.iloc[:,len(LABELS):]
    X_negative = negative.iloc[:,len(LABELS):]
    Y_positive = positive.loc[:,CLASSIFIER]
    Y_negative = negative.loc[:,CLASSIFIER]

    x_train1, x_test1, y_train1, y_test1 = train_test_split(X_positive, Y_positive, test_size = 0.2, random_state = 42, shuffle = True)
    x_train2, x_test2, y_train2, y_test2 = train_test_split(X_negative, Y_negative, test_size = 0.2, random_state = 42, shuffle = True)

    x_train = pd.concat([x_train1, x_train2], axis=0)
    x_test = pd.concat([x_test1, x_test2], axis=0)
    y_train = pd.concat([y_train1, y_train2], axis=0)
    y_test = pd.concat([y_test1, y_test2], axis=0)

    ## LOAD INTO DATA LOADER
    data = load_data(x_train, y_train, x_test, y_test)

    ## CREATE BATCHES
    dummy_batch_gen = batch_generator(data, batch_size=BATCH_SIZE, num_classes=NUM_CLASSES, num_iterations=5e3, seed=42)
    train_batch = next(dummy_batch_gen.gen_train())
    valid_batch, i = next(dummy_batch_gen.gen_valid())
    test_batch, i = next(dummy_batch_gen.gen_test())

    ## BUILD THE MODEL
    net = Net()
    if use_cuda:
        net.cuda()

    ## BUILD THE COST FUNCTION
    criterion = CrossEntropyLoss()

    # weight_decay is equal to L2 regularization
    optimizer = optim.Adam(net.parameters(), lr=LEARNING_RATE, weight_decay = WEIGHT_DECAY)

    ## TRAIN THE MODEL
    # Setup settings for training 
    VALIDATION_SIZE = 0.1 # 0.1 is ~ 100 samples for validation
    log_every = 200
    eval_every = 100

    # Generate batches
    #batch_gen = data_utils.batch_generator(data,
    batch_gen = batch_generator(data,
                               batch_size=BATCH_SIZE,
                               num_classes=NUM_CLASSES,
                               num_iterations=MAX_ITER,
                               seed=42,
                               val_size=VALIDATION_SIZE)

    # Initialize lists for training and validation
    train_iter = []
    train_loss, train_accs = [], []
    valid_iter = []
    valid_loss, valid_accs = [], []

    avg_loss = []
    avg_accs = []

    # Train network
    net.train()
    for i, batch_train in enumerate(batch_gen.gen_train()):
        if i % eval_every == 0:
            # Do the validation
            net.eval()
            val_losses, val_accs, val_lengths = 0, 0, 0
            for batch_valid, num in batch_gen.gen_valid():
                if num != BATCH_SIZE:
                    continue
                output = net(**get_input(batch_valid))
                labels_argmax = torch.max(get_labels(batch_valid), 1)[1]
                val_losses += criterion(output['out'], labels_argmax) * num
                val_accs += accuracy(output['out'], labels_argmax) * num
                val_lengths += num

            # Divide by the total accumulated batch sizes
            val_losses /= val_lengths
            val_accs /= val_lengths
            valid_loss.append(get_numpy(val_losses))
            valid_accs.append(get_numpy(val_accs))
            valid_iter.append(i)
    #         print("Valid, it: {} loss: {:.2f} accs: {:.2f}\n".format(i, valid_loss[-1], valid_accs[-1]))
            net.train()

        # Train network
        output = net(**get_input(batch_train))
        labels_argmax = torch.max(get_labels(batch_train), 1)[1]
        batch_loss = criterion(output['out'], labels_argmax)

        train_iter.append(i)
        train_loss.append(float(get_numpy(batch_loss)))
        train_accs.append(float(get_numpy(accuracy(output['out'], labels_argmax))))

        optimizer.zero_grad()
        batch_loss.backward()
        optimizer.step()
        #scheduler.step()

        # Log i figure
        if i % log_every == 0:

            #clear_output(wait=True)

            avg_loss.append(np.average(train_loss))
            avg_accs.append(np.average(train_accs))

        if MAX_ITER < i:
            break

    train_loss, train_acc = avg_loss[-1], avg_accs[-1]

    ## TEST THE MODEL
    test_accs = []
    cvl_preds, cvl_accs = [], []
    pvl_preds, pvl_accs = [], []
    preds, targs = [], []
    # since we're not training, we don't need to calculate the gradients for our outputs
    with torch.no_grad():
        for batch_test, num in batch_gen.gen_test():
            if num != BATCH_SIZE:
                continue
            output = net(**get_input(batch_test))
            targets = torch.max(get_labels(batch_test), 1)[1]  # 1 is cvl, 0 is pvl
            predictions = torch.max(output['out'], 1)[1]
            pred_isCorrect = torch.eq(predictions, targets)
            batch_acc = torch.mean(pred_isCorrect.float())
            test_accs.append(float(batch_acc.item()))

            pred_accs = torch.max(output['out'], 1)[0]
            pred_accs = pred_accs.tolist()

            # for plotting purposes
            targets = targets.tolist()
            predictions = predictions.tolist()
            targs.extend(targets)
            preds.extend(predictions)

            # for results    
            for i in range(len(targets)):
                if targets[i] == 1:
                    if predictions[i] == 1:
                        cvl_preds.append(True)
                        cvl_accs.append(pred_accs[i])
                    else:
                        cvl_preds.append(False)
                elif targets[i] == 0:
                    if predictions[i] == 0:
                        pvl_preds.append(True)
                        pvl_accs.append(pred_accs[i])
                    else:
                        pvl_preds.append(False)
    
    test_acc = np.average(test_accs)
    error_true = np.average(cvl_preds) # true pos
    no_error_true = np.average(pvl_preds) # true neg
    error_true_accs = np.average(cvl_accs)
    no_error_true_accs = np.average(pvl_accs)

    results = pd.concat([results, pd.DataFrame({'subID':[subID],
                                      'task':['combined'],
                                      'avg_tot_dur':[avg_tot_dur],
                                      'train_loss':[train_loss],
                                      'train_acc':[train_acc],
                                      'test_acc':[test_acc],
                                      'error_true':[error_true],
                                      'no_error_true':[no_error_true],
                                      'error_true_accs':[error_true_accs],
                                      'no_error_true_accs':[no_error_true_accs]})])

    print(f'Test acc {subID}: {test_acc} [true pos = {error_true}, true neg = {no_error_true}]')

Test acc P19: 0.6266666730244954 [true pos = 0.6875, true neg = 0.5571428571428572]
Test acc P24: 0.6921052603345168 [true pos = 0.5, true neg = 0.7991803278688525]
Test acc P6: 0.6250000042574746 [true pos = 0.43609022556390975, true neg = 0.7959183673469388]
Test acc P5: 0.7444444431198968 [true pos = 0.9044117647058824, true neg = 0.25]
Test acc P21: 0.75 [true pos = 0.7241379310344828, true neg = 0.7741935483870968]
Test acc P9: 0.5772727294401689 [true pos = 0.5315315315315315, true neg = 0.6238532110091743]
Test acc P14: 0.7062499951571226 [true pos = 0.8681318681318682, true neg = 0.4927536231884058]
Test acc P15: 0.8333333333333334 [true pos = 0.9565217391304348, true neg = 0.42857142857142855]
Test acc P13: 0.6733333304524421 [true pos = 0.896551724137931, true neg = 0.20618556701030927]
Test acc P18: 0.6727272786877372 [true pos = 0.7902097902097902, true neg = 0.45454545454545453]
Test acc P25: 0.5961538495925757 [true pos = 0.3869565217391304, true neg = 0.7620689655172413]

In [21]:
results.to_csv('/data/Isabella/thesis_spring2022/NN/results_after_FI.csv')

In [22]:
results

,subID,task,avg_tot_dur,train_loss,train_acc,test_acc,error_true,no_error_true,error_true_accs,no_error_true_accs
0,P19,combined,6.201509,0.597313,0.688820,0.626667,0.687500,0.557143,0.962905,0.947082
0,P24,combined,6.062364,0.569662,0.729490,0.692105,0.500000,0.799180,0.926523,0.945131
0,P6,combined,6.704932,0.559025,0.736023,0.625000,0.436090,0.795918,0.921458,0.966606
0,P5,combined,8.287311,0.527193,0.784555,0.744444,0.904412,0.250000,0.979514,0.982248
0,P21,combined,8.479499,0.357770,0.953349,0.750000,0.724138,0.774194,0.997330,0.965248
0,P9,combined,7.457047,0.550240,0.752866,0.577273,0.531532,0.623853,0.970370,0.976538
0,P14,combined,7.237658,0.557704,0.750350,0.706250,0.868132,0.492754,0.986845,0.992951
0,P15,combined,7.027839,0.406700,0.905965,0.833333,0.956522,0.428571,0.998557,0.985639
0,P13,combined,6.855649,0.576228,0.729474,0.673333,0.896552,0.206186,0.980518,0.918382
0,P18,combined,7.451448,0.542691,0.758664,0.672727,0.790210,0.454545,0.962190,0.921216
